In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/MyDrive/directory/WikibaseIntegration"

In [ ]:
!pip install wikidataintegrator --quiet

In [ ]:
import wikidataintegrator as WI
from datetime import datetime
import json, csv, sys, os, requests, re

bot_user='user@bot'
bot_pwd='password'

wikibase_url = "https://medievalcharterskg.wikibase.cloud"
mediawiki_api_url = "https://medievalcharterskg.wikibase.cloud/w/api.php"
login_instance = WI.wdi_login.WDLogin(user=bot_user, pwd=bot_pwd, mediawiki_api_url=mediawiki_api_url)


extension = False

# Properties

In [ ]:
csv_path = './add_properties.csv'
print(f'Using file: {csv_path}')

with open(csv_path, 'r', encoding='latin-1') as csv_file:
    reader = csv.reader(csv_file, delimiter=';')
    headers = next(reader)

    for row in reader:
        property_label_en = row[0]
        datatype = row[1]
        property_description = row[2]
        property_label_es = row[3]

        property = WI.wdi_core.WDItemEngine(mediawiki_api_url=mediawiki_api_url, data=[])
        property.set_label(lang="en", label=property_label_en)
        property.set_label(lang="es", label=property_label_es)

        result = property.write(login=login_instance, entity_type='property', property_datatype=datatype)

        print(f"Created property: {result}")

# Base entities

In [ ]:
csv_path = './items.csv'
complete_data = []
print(f'Using file: {csv_path}')

try:
    with open(csv_path, 'r', encoding='latin-1') as csv_file:
        reader = csv.reader(csv_file, delimiter=';')
        headers = next(reader)

        for row in reader:

            item_label_en = row[0].strip()
            item_label_es = row[1].strip()
            description_en = row[2].strip()

            print(f"Processing item: EN='{item_label_en}', ES='{item_label_es}', Desc='{description_en}'")

            data = []

            for i in range(3, len(row)):
                claim_value = row[i].strip()

                if claim_value:
                    header_parts = headers[i].strip().split(':', 1)

                    if len(header_parts) < 2:
                        print(f"Warning: Skipping column '{headers[i]}' for item '{item_label_en}' - header format should be 'Px:datatype'")
                        continue

                    prop_id = header_parts[0].strip()
                    datatype = header_parts[1].strip()

                    if datatype == 'wikibase-item':
                        # For wikibase-item, the value in the CSV should be a Q-id (e.g., "Q123")
                        if not claim_value.startswith('Q') or not claim_value[1:].isdigit():
                            print(f"Warning: Skipping claim '{prop_id}' for item '{item_label_en}' with value '{claim_value}' - expected a QID for wikibase-item datatype.")
                            continue

                        statement = WI.wdi_core.WDItemID(value=claim_value, prop_nr=prop_id)
                        data.append(statement)

            try:
                item = WI.wdi_core.WDItemEngine(data=data, mediawiki_api_url=mediawiki_api_url)

                if item_label_en:
                    item.set_label(lang="en", label=item_label_en)
                if item_label_es:
                    item.set_label(lang="es", label=item_label_es)

                if description_en:
                    item.set_description(lang="en", description=description_en)
                result = item.write(login=login_instance, bot_account=True)

                if result:
                    print(f"Successfully created item: {item_label_en} ({result})")
                    new_row = row.copy()
                    new_row.append(result)
                    complete_data.append(new_row)
                else:
                    print(f"Error: No QID returned after writing item {item_label_en}. Result: {result}")
                    complete_data.append(row.copy() + ['NO_QID_RETURNED'])

            except Exception as write_error:
                print(f"Error writing item '{item_label_en}': {write_error}")
                complete_data.append(row.copy() + ['WRITE_ERROR'])

except FileNotFoundError:
        print(f"Error: CSV file not found at {csv_path}")
        sys.exit(1)
except ValueError as ve:
      print(f"Configuration Error: {ve}")
      sys.exit(1)
except Exception as e:
    print(f"An unexpected error occurred during CSV processing: {e}")
    sys.exit(1)


# Load entities
## First run: Entities and their Ids

In [ ]:
csv_path = './wikibase_import_dataset1.csv'
ref_prop = 'P63' #'P68'

In [ ]:
def show_message(message):
    active = True
    if active:
      print(message)

def execute_sparql_query(prefix='', query='', endpoint=f'{wikibase_url}/query/sparql',
                         user_agent="User Agent Name"):
    quote_check = re.search("\s'(.*)'\s.", query)
    if quote_check is not None:
        replace_with = quote_check[1].replace("'","\\'")
        query = query.replace(quote_check[1],replace_with)


    params = {
        'query': prefix + '\n#Tool: PBB_core fastrun\n' + query,
        'format': 'json'
    }

    headers = {
        'Accept': 'application/sparql-results+json',
        'User-Agent': user_agent
    }
    response = requests.get(endpoint, params=params, headers=headers)
    response.raise_for_status()

    return response.json()

def searchForItemUsingLabel(base_url,label,language,login_instance):
    item = None
    url = base_url+f"/w/api.php?action=wbsearchentities&search={label}&language={language}&format=json&type=item"

    response = requests.get(url)
    search_results = response.json()

    if search_results['search']:
        for result in search_results['search']:
                if result["match"]["text"] == label:
                    qid = result['id']
                    show_message(f"QID encontrado: {qid}")
                    return qid
    return item

def loadItemUsingLabel(base_url,label,language,login_instance):
    item = None
    url = base_url+f"/w/api.php?action=wbsearchentities&search={label}&language={language}&format=json&type=item"

    response = requests.get(url)
    search_results = response.json()

    if search_results['search']:
        qid = search_results['search'][0]['id']
        show_message(f"QID encontrado: {qid}")

        item = WI.wdi_core.WDItemEngine(wd_item_id=qid, mediawiki_api_url=mediawiki_api_url)

        return item


    return item


def searchEnLabel(label):
  return searchForItemUsingLabel(wikibase_url,label,"en",login_instance)

def loadEnLabel(label):
  return loadItemUsingLabel(wikibase_url,label,"en",login_instance)

def searchEnID(id):
  query = f'''
          PREFIX wd: <{wikibase_url}/entity/>
          PREFIX wdt: <{wikibase_url}/prop/direct/>

          SELECT ?item WHERE {{
            ?item wdt:P1 "{id}" .
          }}
          ORDER BY DESC(xsd:integer(REPLACE(STR(?item), "^.+Q", "")))
          LIMIT 1
          '''
  results = execute_sparql_query(query)['results']['bindings']
  if results:
    return results[0]['item']['value'].split('/')[-1]
  return None

def loadEnID(id):
  qid = searchEnID(id)
  show_message(f"Loading {qid}")
  if qid:
    return WI.wdi_core.WDItemEngine(wd_item_id=qid, mediawiki_api_url=mediawiki_api_url)
  return None

In [ ]:

print(f'Using file: {csv_path}')

complete_data = []

with open(csv_path, 'r', encoding='latin-1') as csv_file:
    reader = csv.reader(csv_file, delimiter=';')
    headers = next(reader)

    for row in reader:
        item_label = row[0]
        description = row[1]
        alias = row[2]

        if extension:
          if row[4]:
              wd_item = None
              id_index = 0
              id_list = row[4].split('|')
              while wd_item is None and id_index  < len(id_list):
                first_id = id_list[id_index]
                wd_item = searchEnID(first_id)
                id_index += 1
              if wd_item:
                  print(f"Item with ID {first_id} already exists, skipping.")
                  continue
          elif searchEnLabel(item_label):
              print(f"Item with label '{item_label}' already exists. Skipping.")
              continue

        statements = []

        for i in range(2, len(row)):
            if row[i] and row[i].strip():
                prop_info = headers[i].split(':')
                prop_id = prop_info[0]
                prop_type = prop_info[1] if len(prop_info) > 1 else ""

                if prop_id == "instance of":
                    statement = WI.wdi_core.WDItemID(value=row[i], prop_nr="P3")
                    statements.append(statement)
                elif prop_id == "P1" and prop_type == "string" :
                    value = row[i]
                    if row[3] in ["Q39", "Q56"]:
                        all_ids = value.split('|')
                        for id in all_ids:
                            statement = WI.wdi_core.WDString(value=id, prop_nr=prop_id)
                            statements.append(statement)
                    else:
                      statement = WI.wdi_core.WDString(value=value, prop_nr=prop_id)
                      statements.append(statement)

        try:
            new_item = WI.wdi_core.WDItemEngine(
                data=statements,
                mediawiki_api_url=mediawiki_api_url
            )

            new_item.set_label(item_label, lang='en')

            new_item.set_description(description, lang='en')

            if alias.strip():
                new_item.set_aliases([alias], lang='en')

            result = new_item.write(login_instance, bot_account=True)

            new_row = row.copy()
            new_row.append(result)

            print(result, item_label)
            complete_data.append(new_row)

        except Exception as e:
            if 'already exists' in str(e):
                print(f"Warning: Item with label '{item_label}' already exists. Skipping.")
            else:
                print(f"Error creating item '{item_label}': {str(e)}")


# Second run: add properties

In [ ]:
document_mapping = {}

with open(csv_path, 'r', encoding='latin-1') as csv_file:
    reader = csv.reader(csv_file, delimiter=';')
    headers = next(reader)
    data = []

    for row in reader:
      if row[1] in ["Instance of Purchase", "Instance of Acquisition", "Instance of Activity", "Instance of Production"]:
        DID = row[0].split('-')[1].split('_')[0]
        document = row[35]
        document_mapping[DID] = document

print(document_mapping)

In [ ]:
def search_statement(data, prop_nr, value, value_type="item"):
    for stmt in data:
        if value_type == "item":
            search_value = f"Q{stmt.get_value()}"
        elif value_type == "quantity" and isinstance(stmt.get_value(), tuple):
            search_value = stmt.get_value()[0].strip('+')
        else:
            search_value = stmt.get_value()
        if stmt.prop_nr == prop_nr and search_value == value:
            return stmt, True

    return None, False

def add_new_reference(statement):
  exists = False
  for ref_group in statement.references:
     _, exists = search_statement(ref_group, ref_prop, "Q55")
     if exists:
       break
  if not exists:
    reference = WI.wdi_core.WDItemID(value="Q55", prop_nr=ref_prop, is_reference=True)
    statement.references.append([reference])
    show_message(f"Added reference {ref_prop} -> Q55")

def add_statement_1r(data, single_value, PID, target_qid):
  statement, exists = search_statement(data, PID, target_qid)

  if exists:
      if extension:
        add_new_reference(statement)
  else:
      statement = WI.wdi_core.WDItemID(value=target_qid, prop_nr=PID)
      reference = WI.wdi_core.WDItemID(value="Q55", prop_nr=ref_prop, is_reference=True)
      statement.references.append([reference])
      show_message(f"  Added property {PID} with value '{single_value}' (QID {target_qid}) and reference P63 -> Q55")
      data.append(statement)

def add_statement_1q(data, single_value, PID, target_qid, target_pqid):
  statement, exists = search_statement(data, PID, target_qid)

  if exists:
      qualifier = WI.wdi_core.WDItemID(value=target_pqid, prop_nr="P28", is_qualifier=True)
      statement.qualifiers.append(qualifier)
      if extension:
        add_new_reference(statement)
      show_message(f"  Added qualifier P28 -> {target_pqid} to property {PID} with value '{target_qid}'")
  else:
      statement = WI.wdi_core.WDItemID(value=target_qid, prop_nr=PID)
      reference = WI.wdi_core.WDItemID(value="Q55", prop_nr=ref_prop, is_reference=True)
      qualifier_2 = WI.wdi_core.WDItemID(value=target_pqid, prop_nr="P28", is_qualifier=True)
      statement.references.append([reference])
      statement.qualifiers.append(qualifier_2)
      data.append(statement)
      show_message(f"  Added property {PID} with value '{single_value}' (QID {target_qid}), qualifier P28 -> {target_pqid} and reference {ref_prop} -> Q55")

def add_statement_2q(data, single_value, PID, target_qid, target_pqid, target_pqid2, propq):
  statement, exists = search_statement(data, PID, target_qid)

  if exists:
      qualifier = WI.wdi_core.WDItemID(value=target_pqid, prop_nr="P28", is_qualifier=True)
      qualifier_3 = WI.wdi_core.WDItemID(value=target_pqid2, prop_nr=propq, is_qualifier=True)
      statement.qualifiers.append(qualifier)
      statement.qualifiers.append(qualifier_3)
      if extension:
        add_new_reference(statement)
      show_message(f"  Added qualifier P28 -> {target_pqid} to property {PID} with value '{target_qid}' and  qualifier {propq} -> {target_pqid2}'")
  else:
      statement = WI.wdi_core.WDItemID(value=target_qid, prop_nr=PID)
      reference = WI.wdi_core.WDItemID(value="Q55", prop_nr=ref_prop, is_reference=True)
      qualifier_2 = WI.wdi_core.WDItemID(value=target_pqid, prop_nr="P28", is_qualifier=True)
      qualifier_3 = WI.wdi_core.WDItemID(value=target_pqid2, prop_nr=propq, is_qualifier=True)
      statement.references.append([reference])
      statement.qualifiers.append(qualifier_2)
      statement.qualifiers.append(qualifier_3)
      data.append(statement)
      show_message(f"  Added property {PID} with value '{single_value}' (QID {target_qid}), qualifier P28 -> {target_pqid}, qualifier {propq} -> {target_pqid2} and reference {ref_prop} -> Q55")

def add_statement_1q_string(data, single_value, PID, target_pqid):
  statement, exists = search_statement(data, PID, single_value, "string")

  if exists:
      qualifier = WI.wdi_core.WDItemID(value=target_pqid, prop_nr="P28", is_qualifier=True)
      statement.qualifiers.append(qualifier)
      if extension:
        add_new_reference(statement)
      show_message(f"  Added qualifier P28 -> {target_pqid} to property {PID} with value '{single_value}'")
  else:
      statement = WI.wdi_core.WDString(value=single_value, prop_nr=PID)
      qualifier_1 = WI.wdi_core.WDItemID(value="Q55", prop_nr=ref_prop, is_reference=True)
      qualifier_2 = WI.wdi_core.WDItemID(value=target_pqid, prop_nr="P28", is_qualifier=True)
      statement.references.append([qualifier_1])
      statement.qualifiers.append(qualifier_2)
      data.append(statement)
      show_message(f"  Added property {PID} with value '{single_value}', qualifier P28 -> {target_pqid} and references {ref_prop} -> Q55")

def add_statement_1r_string(data, single_value, PID):
  statement, exists = search_statement(data, PID, single_value, "string")

  if exists:
      if extension:
        add_new_reference(statement)
  else:
      statement = WI.wdi_core.WDString(value=single_value, prop_nr=PID)
      reference = WI.wdi_core.WDItemID(value="Q55", prop_nr=ref_prop, is_reference=True)
      statement.references.append([reference])
      show_message(f"  Added property {PID} with value '{single_value}' and reference {ref_prop} -> Q55")
      data.append(statement)

def add_statement_1r_quantity(data, single_value, PID):
  statement, exists = search_statement(data, PID, single_value, "quantity")

  if exists:
      if extension:
        add_new_reference(statement)
  else:
      statement = WI.wdi_core.WDQuantity(value=single_value, prop_nr=PID)
      reference = WI.wdi_core.WDItemID(value="Q55", prop_nr=ref_prop, is_reference=True)
      statement.references.append([reference])
      show_message(f"  Added property {PID} with value '{single_value}' and reference {ref_prop} -> Q55")
      data.append(statement)

def search_native_label(label):
  query = f'''
          PREFIX wd: <{wikibase_url}/entity/>
          PREFIX wdt: <{wikibase_url}/prop/direct/>

          SELECT ?item WHERE {{
            ?item wdt:P69 "{label}".
          }}
          ORDER BY DESC(xsd:integer(REPLACE(STR(?item), "^.+Q", "")))
          LIMIT 1
          '''
  results = execute_sparql_query(query)['results']['bindings']
  if results: #This person has an entity name assigned
    return results[0]['item']['value'].split('/')[-1]
  return None

def add_name_label(data, person_id, name_label, name_pid):
  query = f'''
          PREFIX wd: <{wikibase_url}/entity/>
          PREFIX wdt: <{wikibase_url}/prop/direct/>

          SELECT ?name WHERE {{
            ?item wdt:P1 "{person_id}";
                  wdt:{name_pid} ?name.
          }}
          ORDER BY DESC(xsd:integer(REPLACE(STR(?item), "^.+Q", "")))
          LIMIT 1
          '''
  results = execute_sparql_query(query)['results']['bindings']
  doc_qid = searchEnLabel(document_mapping[name_label.split("-")[0]])
  name = name_label.split("-")[1]
  if results: #This person has an entity name assigned
    name_qid = results[0]['name']['value'].split('/')[-1]
    wd_item_to_update = WI.wdi_core.WDItemEngine(wd_item_id=name_qid, mediawiki_api_url=mediawiki_api_url)
    data = wd_item_to_update.original_statements
    add_statement_1q_string(data, name, "P69", doc_qid)
    wd_item_to_update = WI.wdi_core.WDItemEngine(wd_item_id=name_qid, data=data, mediawiki_api_url=mediawiki_api_url)
    r = wd_item_to_update.write(login_instance)
    show_message(f"Updated succesfully name: {r}")
  else:
    #Does it exist?
    name_qid = searchEnLabel(name)
    person_qid = searchEnID(person_id)
    if name_qid:
      name_item = WI.wdi_core.WDItemEngine(wd_item_id=name_qid, mediawiki_api_url=mediawiki_api_url)
      data = name_item.original_statements
      add_statement_1q_string(data, name, "P69", doc_qid)
      name_item = WI.wdi_core.WDItemEngine(wd_item_id=name_qid, data=data, mediawiki_api_url=mediawiki_api_url)
      r = name_item.write(login_instance)
      show_message(f"Updated entity name: {name_qid}")
    else:
      statements = []
      iof = "Q53" if name_pid == "P49" else "Q54"
      statement_1 = WI.wdi_core.WDItemID(value=iof, prop_nr="P3")
      statement_2 = WI.wdi_core.WDString(value=name, prop_nr="P69")
      qualifier = WI.wdi_core.WDItemID(value=doc_qid, prop_nr="P28", is_qualifier=True)
      reference = WI.wdi_core.WDItemID(value="Q55", prop_nr=ref_prop, is_reference=True)
      statement_2.qualifiers.append(qualifier)
      statement_2.references.append([reference])
      statements.append(statement_1)
      statements.append(statement_2)
      new_name = WI.wdi_core.WDItemEngine(
          data=statements,
          mediawiki_api_url=mediawiki_api_url
      )

      new_name.set_label(name, lang='en')

      description = "Given name" if name_pid == "P49" else "Family name"

      new_name.set_description(description, lang='en')

      alias = f"{name} (given name)" if name_pid == "P49" else f"{name} (family name)"

      if alias.strip():
          new_name.set_aliases([alias], lang='en')

      name_qid = new_name.write(login_instance, bot_account=True)
      show_message(f"Created new entity name: {name_qid}")

    statement = WI.wdi_core.WDItemID(value=name_qid, prop_nr=name_pid)
    reference = WI.wdi_core.WDItemID(value="Q55", prop_nr=ref_prop, is_reference=True)
    statement.references.append([reference])

    wd_item_to_update = WI.wdi_core.WDItemEngine(wd_item_id=person_qid, mediawiki_api_url=mediawiki_api_url)
    wd_item_to_update.update(data=[statement])
    r = wd_item_to_update.write(login_instance)
    show_message(f"Updated person {person_qid} with name {name}")


In [ ]:
with open(csv_path, 'r', encoding='latin-1') as csv_file:
    reader = csv.reader(csv_file, delimiter=';')
    headers = next(reader)
    complete_data = []
    errors_data = []
    update = False

    if update:
      for row in reader:
        try:

            if row[3] == "Q16": #Person
                  wd_item_to_update = loadEnID(row[2])
            else:
                  wd_item_to_update = loadEnLabel(row[0])

            show_message("Removing statements...")
            for statement in wd_item_to_update.statements:
                if statement.prop_nr != "P1" and statement.prop_nr != "P3":
                    WI.wdi_core.WDItemEngine.delete_statement(statement.get_id(),wd_item_to_update.lastrevid, login_instance, mediawiki_api_url=mediawiki_api_url)

        except Exception as e:
            print("There was an error with this one, skipping:")
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)
            print(row)
            print(e)
            errors_data.append(row)

    csv_file.seek(0)

    reader = csv.reader(csv_file, delimiter=';')
    headers = next(reader)
    for row in reader:
        data = []
        try:
          if extension:
            if row[4]:
              original_entity = None
              id_index = 0
              id_list = row[4].split('|')
              while original_entity is None and id_index  < len(id_list):
                first_id = id_list[id_index]
                original_entity = loadEnID(first_id)
                id_index += 1
              data = original_entity.original_statements
            else:
              data = loadEnLabel(row[0]).original_statements

          item_label = row[0]
          show_message(f"Processing ({item_label})...")


          for i in range(4, len(row)):  # Skip label, description, instance of, ID
              if i >= len(headers) or not row[i] or not row[i].strip():
                  continue

              if ':' not in headers[i]:
                  continue

              PID = headers[i].split(':')[0]
              prop_type = headers[i].split(':')[1]

              value = row[i].strip()

              # Check if the value contains pipe-separated values
              values = [v.strip() for v in value.split('|')]

              dp_pattern = r'^D\d+P\d+$'
              doc_pattern = r'^D\d+Oc\d+$'
              dpb_pattern = r'^D\d+PB\d+$'
              dt_pattern = r'^D\d+-.+$'
              d_pattern = r'^D\d+$'
              pq_pattern = r'^wikibase-itemPQ\d+-[a-zA-Z-]+$'

              for single_value in values:
                  if not single_value:
                      continue
                  else:
                      if prop_type == "wikibase-item":
                          if re.match(dp_pattern, single_value):  #D1P1
                              ids = single_value.split('P')
                              document_id = ids[0]
                              person_id = f"AMSPO-P{ids[1]}"
                              target_qid = searchEnID(person_id)
                              target_pqid = searchEnLabel(document_mapping[document_id])
                              add_statement_1q(data, single_value, PID, target_qid, target_pqid)
                          elif re.match(dp_pattern, single_value):  #D1Oc1
                              ids = single_value.split('Oc')
                              document_id = ids[0]
                              occupation_id = f"AMSPO-{single_value}"
                              target_qid = searchEnID(occupation_id)
                              target_pqid = searchEnLabel(document_mapping[document_id])
                              add_statement_1q(data, single_value, PID, target_qid, target_pqid)
                          elif re.match(dpb_pattern, single_value): #D1PB1
                              document_id = single_value.split('P')[0]
                              person_id = f"AMSPO-{single_value}"
                              target_qid = searchEnID(person_id)
                              target_pqid = searchEnLabel(document_mapping[document_id])
                              add_statement_1q(data, person_id, PID, target_qid, target_pqid)
                          elif re.match(dt_pattern, single_value):
                              ids = single_value.split('-')
                              document_id = ids[0]
                              label = ids[1]
                              if PID in ["P53", "P55"]: #D1-Oviedo
                                target_qid = searchEnID(label)
                                target_pqid = searchEnLabel(document_mapping[document_id])
                                add_statement_1q(data, single_value, PID, target_qid, target_pqid)
                              elif extension and PID in ["P49", "P50"]: #Names with old transcriptions of existing entities
                                add_name_label(data, first_id, single_value, PID)
                              else:
                                target_qid = searchEnLabel(label) #D1-Orfebre
                                if target_qid is None:
                                  target_qid = search_native_label(label)
                                target_pqid = searchEnLabel(document_mapping[document_id])
                                add_statement_1q(data, single_value, PID, target_qid, target_pqid)
                          elif re.match(d_pattern, single_value): #D1
                              target_qid = searchEnLabel(document_mapping[single_value])
                              add_statement_1r(data, single_value, PID, target_qid)
                          else:
                              if PID in ["P27", "P64", "P35"]:
                                target_qid = searchEnID(single_value) #Borondes_Arriba, D1Death
                              else:
                                target_qid = searchEnLabel(single_value) #Maravedi
                              add_statement_1r(data, single_value, PID, target_qid)
                      elif re.match(pq_pattern, prop_type): #P61:wikibase-itemPQ62-Grandchild
                          if re.match(dp_pattern, single_value): #D1P1
                            ids = single_value.split('P')
                            document_id = ids[0]
                            person_id = f"AMSPO-P{ids[1]}"
                          elif re.match(dpb_pattern, single_value): #D1PB1
                            document_id = single_value.split('P')[0]
                            person_id = f"AMSPO-{single_value}"
                          target_qid = searchEnID(person_id)
                          target_pqid = searchEnLabel(document_mapping[document_id])
                          fxql = prop_type.split('PQ')[1].split('-', 1)
                          propq = f"P{fxql[0]}"
                          target_pqid2 = searchEnLabel(fxql[1])
                          add_statement_2q(data, single_value, PID, target_qid, target_pqid, target_pqid2, propq)
                      elif prop_type == "string" and PID != "P1":
                          if re.match(dt_pattern, single_value): #D1-Vasallo
                              ids = single_value.split('-')
                              document_id = ids[0]
                              value = ids[1]
                              target_pqid = searchEnLabel(document_mapping[document_id])
                              add_statement_1q_string(data, value, PID, target_pqid)
                          else: #Alfonso
                              add_statement_1r_string(data, single_value, PID)
                      elif prop_type == "quantity":
                          add_statement_1r_quantity(data, single_value, PID)
                      elif prop_type == "time":
                          date_parts = single_value.split('-')
                          year = int(date_parts[0])
                          month = int(date_parts[1])
                          day = int(date_parts[2])

                          date = datetime(year, month, day, 0, 0, 0, 0)
                          iso_date = "+" + date.isoformat() + "Z"

                          statement = WI.wdi_core.WDTime(time=iso_date, prop_nr=PID, timezone=1)

                          show_message(f"  Added property {PID} with value '{single_value}'")
                          data.append(statement)

        except Exception as e:
                    print("There was an error with this one, skipping, error:")
                    print(row)
                    print(e)
                    errors_data.append(row)
                    data = "error"


        if data == 'error' or data == []:
            continue

        try:
          if row[4]:
                wd_item = None
                id_index = 0
                id_list = row[4].split('|')
                while wd_item is None and id_index  < len(id_list):
                  first_id = id_list[id_index]
                  wd_item = searchEnID(first_id)
                  id_index += 1
          else:
                wd_item = searchEnLabel(row[0])

          wd_item_to_update = WI.wdi_core.WDItemEngine(wd_item_id=wd_item, data=data, mediawiki_api_url=mediawiki_api_url)
          r = wd_item_to_update.write(login_instance)

          show_message(f"Updated succesfully: {r}, {row[0]}")

          complete_data.append(row)

        except Exception as e:
            print("There was an error with this one, skipping:")
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)
            print(row)
            print(e)
            errors_data.append({'row': row, 'error': e})


csv_file.close()

In [ ]:
for error in errors_data:
  print(error)

In [ ]:
with open("./wikidata_mappings.csv", 'r', encoding='latin-1') as csv_file:
  reader = csv.reader(csv_file, delimiter=';')
  headers = next(reader)
  for row in reader:
    try:
      id = searchEnID(row[0])
      wik_id = row[1]
      statement = WI.wdi_core.WDString(value=wik_id, prop_nr="P2")
      print(row[0], id)
      wd_item = WI.wdi_core.WDItemEngine(wd_item_id=id, data=[statement], mediawiki_api_url=mediawiki_api_url)
      r = wd_item.write(login_instance)
      print(f"Updated {id} with Wikidata external ID {wik_id}")
    except Exception as e:
            print("There was an error with this one, skipping:")
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)
            print(row)
            print(e)
            errors_data.append({'row': row, 'error': e})
  csv_file.close()